# ** Data Loading and Visualization **

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
from tqdm import tqdm
import sklearn
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings(action='ignore')


import geopandas as gpd
from shapely.geometry import Point, Polygon

In [ ]:
filename = '../input/birdclef-2021/train_short_audio/acafly/XC131193.ogg'

In [ ]:
plt.figure(figsize=(18, 5))

# by default librosa.load returns a sample rate of 22050
# librosa converts input to mono, hence always 
data, sample_rate = librosa.load(filename)
librosa.display.waveplot(data, sr=sample_rate)
print("Sample Rate: ", sample_rate)
ipd.Audio(filename)

In [ ]:
y, sr = librosa.load(filename,sr=32000, offset=None, duration=15)

print('y:', y, '\n')
print('y shape:', np.shape(y), '\n')
print('Sample Rate (KHz):', sr, '\n')

# Verify length of the audio
print('Check Len of Audio:', np.shape(y)[0]/sr)

In [ ]:
# Trim leading and trailing silence from an audio signal (silence before and after the actual audio)
audio_acafly, _ = librosa.effects.trim(y)

# the result is an numpy ndarray
print('Audio File:', audio_acafly, '\n')
print('Audio File shape:', np.shape(audio_acafly))

# Sound waves

In [ ]:
plt.figure(figsize=(14,8))
plt.title('Sound Waves', fontsize=16)

librosa.display.waveplot(audio_acafly, sr , color = "#A300F9")

# Fourier Transform

In [ ]:
n_fft = 2048 
hop_length = 512 

D_acafly= np.abs(librosa.stft(audio_acafly, n_fft = n_fft, hop_length = hop_length))
print('Shape of D object:', np.shape(D_acafly))


# Spectogram

In [ ]:
DB_acafly = librosa.amplitude_to_db(D_acafly, ref = np.max)

plt.figure(figsize=(14,8))
plt.title('Spectrogram', fontsize=16)

librosa.display.specshow(DB_acafly, sr=32000 , hop_length = hop_length,
                         x_axis = 'time', y_axis = 'log', cmap = 'cool')



# Mel Spectrogram

In [ ]:
S_acafly = librosa.feature.melspectrogram(y, sr)
S_DB_acafly = librosa.amplitude_to_db(S_acafly, ref=np.max)

plt.figure(figsize=(14,8))
plt.title('Mel Spectrogram', fontsize=16)

librosa.display.specshow(S_DB_acafly, sr=32000 , hop_length = hop_length,
                         x_axis = 'time', y_axis = 'log', cmap = 'cool')

# Zero Crossing Rate

In [ ]:
zero_acafly = librosa.zero_crossings(audio_acafly, pad=False)
print("acafly change rate is {}".format(sum(zero_acafly)))

# Harmonics and Perceptrual

In [ ]:
y_harm_acafly, y_perc_acafly = librosa.effects.hpss(audio_acafly)

plt.figure(figsize = (16, 6))
plt.plot(y_perc_acafly, color = '#FFB100')
plt.plot(y_harm_acafly, color = '#A300F9')
plt.legend(("Perceptrual", "Harmonics"))
plt.title("Harmonics and Perceptrual : Acafly Bird", fontsize=16);


# Spectral Centroid

In [ ]:
# Calculate the Spectral Centroids
spectral_centroids = librosa.feature.spectral_centroid(audio_acafly, sr=sr)[0]

# Shape is a vector
#print('Centroids:', spectral_centroids, '\n')
print('Shape of Spectral Centroids:', spectral_centroids.shape, '\n')

# Computing the time variable for visualization
frames = range(len(spectral_centroids))

# Converts frame counts to time (seconds)
t = librosa.frames_to_time(frames)

print('frames:', frames, '\n')
#print('t:', t)

# Function that normalizes the Sound Data
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)


In [ ]:
#Plotting the Spectral Centroid along the waveform
plt.figure(figsize = (16, 6))
librosa.display.waveplot(audio_acafly, sr=sr, alpha=0.4, color = '#A300F9', lw=3)
plt.plot(t, normalize(spectral_centroids), color='#FFB100', lw=2)
plt.legend(["Spectral Centroid", "Wave"])
plt.title("Spectral Centroid: Acafly Bird", fontsize=16);

# Chroma Frequencies

In [ ]:
# Increase or decrease hop_length to change how granular you want your data to be
hop_length = 5000

# Chromogram Vesspa
chromagram = librosa.feature.chroma_stft(audio_acafly, sr=sr, hop_length=hop_length)
print('Chromogram Vesspa shape:', chromagram.shape)

plt.figure(figsize=(16, 6))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='twilight')

plt.title("Chromogram: Acafly", fontsize=16);

In [ ]:
tempo_acafly, _ = librosa.beat.beat_track(y, sr = sr)
print("BPM for Acafly is {}".format(tempo_acafly))

# Spectral Rolloff

In [ ]:
# Spectral RollOff Vector
spectral_rolloff = librosa.feature.spectral_rolloff(audio_acafly, sr)[0]

# Computing the time variable for visualization
frames = range(len(spectral_rolloff))
# Converts frame counts to time (seconds)
t = librosa.frames_to_time(frames)

# The plot
plt.figure(figsize = (16, 6))
librosa.display.waveplot(audio_acafly, sr=sr, alpha=0.4, color = '#A300F9', lw=3)
plt.plot(t, normalize(spectral_rolloff), color='#FFB100', lw=3)
plt.legend(["Spectral Rolloff", "Wave"])
plt.title("Spectral Rolloff: Acafly Bird", fontsize=16);